In [16]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas as pd
import numpy as np
import sys
from evaluation.metric import computeAccuracy, computeF1Score, computeAUC, computeAUCBehavior
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline
from sklearn.grid_search import RandomizedSearchCV, GridSearchCV
from scripts.gridsearch import text_grid
from sklearn.externals import joblib


0.17


In [17]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
# clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
clf_behavior_initial = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pd.read_csv("./data/alcohol_training_instances.csv")

Prepare the behavior level data

In [18]:
# appending labels based on whether tweet label contains phrase
behavior_labels = []
behavior_index_in_df = []

fpl = "'first_person_level'"

# extracting FP levels and constructing subset vector simultaneously
for i in range(len(df)):
    if fpl in df.labels[i]:
        behavior_labels.append(int(df.labels[i][df.labels[i].find(fpl) + len(fpl) + 2]))
        behavior_index_in_df.append(1)
    else:
        behavior_index_in_df.append(0)

behavior_labels = list(map(int, behavior_labels))
print(behavior_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(behavior_labels)) # 6357
print(behavior_index_in_df[:20])
print('length of first person level index in df: ', len(behavior_index_in_df)) # 15651
index_behavior = list(np.where(behavior_index_in_df)[0])
print(index_behavior[:15])
print('length of first person level df index: ', len(index_behavior)) # 6357
df_behavior = df.iloc[index_behavior,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# change label 3 to label 0
behavior_labels = [0 if x == 3 else x for x in behavior_labels] 

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_behavior, behavior_labels, test_size=0.2, random_state=26)

[0, 0, 2, 2, 1, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0]
length of first person level labels:  6357
[0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0]
length of first person level index in df:  15651
[2, 5, 6, 7, 11, 13, 17, 18, 21, 22, 23, 24, 26, 27, 28]
length of first person level df index:  6357


In [19]:
print(len(behavior_labels))
print(len(behavior_index_in_df))
# behavior_labels[:30]
# for i in behavior_labels:
#     if i ==3:
#         print(True)

6357
15651


Create a model for the following grid search

In [20]:
# RF
clf_behavior_RF = AlcoholPipeline(global_features=["text"]).pipeline(RandomForestClassifier())
clf_behavior_RF.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf',
  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False))]

In [21]:
clf_behavior_RF.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

Now we are trying to implement grid search.

In [22]:
# clf_behavior_RF.get_params()

In [23]:
param_grid = {
#     'clf__C': uniform(0.0001, 1000),
#     'clf__penalty': ['l2', "l1"],
#     'clf__penalty': ['l2'],
#     'clf__tol': uniform(0.0001, 0.001),
#     'clf__verbose': [0]
#     'features__text__tfidf__analyzer': ['char'],
#     'features__text__tfidf__ngram_range': [(2,5)]
}

param_grid.update(text_grid)

cv_kwargs = dict(
    n_iter=50,
    scoring=None,
    fit_params=None,
    n_jobs=-1,
    iid=True,
    refit=True,
    cv=10,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=0
)

In [24]:
clf_behavior_RF_grid = RandomizedSearchCV(clf_behavior_RF, param_grid, **cv_kwargs)
clf_behavior_RF_grid.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


RandomizedSearchCV(cv=10, error_score=0,
          estimator=Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params={}, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'features__text__tfidf__analyzer': ['char', 'word'], 'features__text__tfidf__lowercase': [False, True], 'features__text__tfidf__max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025BAE1609B0>, 'features__text__tfidf__min_df': <scipy.stats._distn_infrast..., 'english'], 'features__text__tfidf__tokenizer': [None, <function tokenize at 0x0000025BAE220EA0>]},
          pre_dispatch='2*n_jobs', random_state=None

In [25]:
# get the best estimator 
best_estimator_RF = clf_behavior_RF_grid.best_estimator_
print('best estimator: ', best_estimator_RF)
print('-------------------------- mean accuracy on the cross validation of best estimator --------------------------')
print('best score: ', clf_behavior_RF_grid.best_score_)
print('best score related parameters: ', clf_behavior_RF_grid.best_params_)

best estimator:  Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
-------------------------- mean accuracy on the cross validation of best estimator --------------------------
best score:  0.6033431661750246
best score related parameters:  {'features__text__tfidf__analyzer': 'word', 'features__text__tfidf__lowercase': True, 'features__text__tfidf__max_features': 86709, 'features__text__tfidf__min_df': 1, 'features__text__tfidf__ngram_range': (2, 5), 'features__text__tfidf__norm': 'l2', 'features__text__tfidf__stop_words': None, 'features__text__tfidf__tokenizer': <function tokenize at 0x0000025BAE220EA0>}


Test the best estimator performance on testing dataset

In [26]:
# predict in testing set.
y_behavior_RF_grid = best_estimator_RF.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_RF_grid, y_test)
f1_score_RF_grid = computeF1Score(y_test, y_behavior_RF_grid, average = None)
print('F1 Score: ', f1_score_RF_grid)
auc_score_RF_grid = computeAUCBehavior(best_estimator_RF, X_test, y_test, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('RF classifier AUC: ', auc_score_RF_grid)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1272
511 different item in whole dataset
accuracy: 59.827044%
F1 Score:  [0.70996785 0.50080775 0.29189189]
RF classifier AUC:  {'current': 0.729341888493887, 'looking': 0.7399994451414426, 'reflecting': 0.6819647912380304, 'micro': 0.7646731982516515, 'macro': 0.7173064826593593}


Save the best estimator

In [27]:
joblib.dump(best_estimator_RF, './alcohol classifiers/behaviorBestEstimatorRF.pkl', compress=1)

['./alcohol classifiers/behaviorBestEstimatorRF.pkl']

Plot the grid search result in table. 

In [28]:
cv = pd.DataFrame(clf_behavior_RF_grid.grid_scores_)
cv.cv_validation_scores[0]
cv

,parameters,mean_validation_score,cv_validation_scores
0,"{'features__text__tfidf__analyzer': 'char', 'f...",0.597640,"[0.6019607843137255, 0.6019607843137255, 0.589..."
1,"{'features__text__tfidf__analyzer': 'char', 'f...",0.592527,"[0.596078431372549, 0.6098039215686275, 0.5579..."
2,"{'features__text__tfidf__analyzer': 'char', 'f...",0.591150,"[0.6215686274509804, 0.5941176470588235, 0.565..."
3,"{'features__text__tfidf__analyzer': 'char', 'f...",0.597050,"[0.6313725490196078, 0.6058823529411764, 0.581..."
4,"{'features__text__tfidf__analyzer': 'word', 'f...",0.559489,"[0.5431372549019607, 0.5450980392156862, 0.559..."
5,"{'features__text__tfidf__analyzer': 'word', 'f...",0.547296,"[0.5372549019607843, 0.5392156862745098, 0.542..."
6,"{'features__text__tfidf__analyzer': 'word', 'f...",0.603343,"[0.6137254901960785, 0.5941176470588235, 0.612..."
7,"{'features__text__tfidf__analyzer': 'word', 'f...",0.586234,"[0.5882352941176471, 0.5980392156862745, 0.593..."
8,"{'features__text__tfidf__analyzer': 'word', 'f...",0.554376,"[0.5176470588235295, 0.5568627450980392, 0.573..."
9,"{'features__text__tfidf__analyzer': 'word', 'f...",0.587807,"[0.5980392156862745, 0.5862745098039216, 0.587..."


Test the one you are saved

In [29]:
clf_saved = joblib.load('./alcohol classifiers/behaviorBestEstimatorRF.pkl')

In [30]:
# predict in testing set.
y_behavior_RF_saved = clf_saved.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_behavior_RF_saved, y_test)
f1_score_RF_saved = computeF1Score(y_test, y_behavior_RF_saved, average = None)
print('F1 Score: ', f1_score_RF_saved)
auc_score_RF_saved = computeAUCBehavior(clf_saved, X_test, y_test, plot=False, RF=True)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('RF classifier AUC: ', auc_score_RF_saved)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1272
511 different item in whole dataset
accuracy: 59.827044%
F1 Score:  [0.70996785 0.50080775 0.29189189]
RF classifier AUC:  {'current': 0.729341888493887, 'looking': 0.7399994451414426, 'reflecting': 0.6819647912380304, 'micro': 0.7646731982516515, 'macro': 0.7173064826593593}


Saved model works well